# Tidy-Data

In [1]:
# libraries
import numpy as np
import pandas as pd
import altair as alt
from sklearn.decomposition import PCA

# warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# display settings
pd.options.display.max_columns = None

### Raw data

**First, convert the raw .xlsx file to a .csv file and then use '.read_csv'**

In [2]:
data = pd.read_csv('data/walkable_distance_to_public_transportation.csv', low_memory = False)
data.head(3)

,ind_id,ind_definition,reportyear,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,region_name,region_code,pop_trans_acc,pop2010,p_trans_acc,LL_95CI,UL_95CI,se,rse,SAC_decile,SAC_RR,version,MTC_decile,MTC_RR,SD_decile,SD_RR,SC_decile,SC_RR
0,51,Percent of population residing within ½ mile o...,2008.0,3.0,AfricanAm,CO,6061.0,Placer,Placer,6061.0,Sacramento Area,8.0,55.0,4427.0,0.012424,0.009161,0.015687,0.001665,13.399974,NaN,0.070460,26Nov2013,NaN,NaN,NaN,NaN,NaN,NaN
1,51,Percent of population residing within ½ mile o...,2008.0,1.0,AIAN,CO,6061.0,Placer,Placer,6061.0,Sacramento Area,8.0,51.0,2080.0,0.024519,0.017873,0.031166,0.003391,13.830066,NaN,0.139059,26Nov2013,NaN,NaN,NaN,NaN,NaN,NaN
2,51,Percent of population residing within ½ mile o...,2008.0,2.0,Asian,CO,6061.0,Placer,Placer,6061.0,Sacramento Area,8.0,117.0,19963.0,0.005861,0.004802,0.006920,0.000540,9.217872,NaN,0.033239,26Nov2013,NaN,NaN,NaN,NaN,NaN,NaN


### Cleaning the data

**data_mod1:** removing the last row

In [3]:
# Last row is the 'END OF TABLE' row with only 'NaN' values
data_mod1 = data.copy()
data_mod1.tail(1)

,ind_id,ind_definition,reportyear,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,region_name,region_code,pop_trans_acc,pop2010,p_trans_acc,LL_95CI,UL_95CI,se,rse,SAC_decile,SAC_RR,version,MTC_decile,MTC_RR,SD_decile,SD_RR,SC_decile,SC_RR
66009,END OF TABLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Last row should now have index '66008'
data_mod1 = data_mod1[:-1]
data_mod1.tail(1)

,ind_id,ind_definition,reportyear,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,region_name,region_code,pop_trans_acc,pop2010,p_trans_acc,LL_95CI,UL_95CI,se,rse,SAC_decile,SAC_RR,version,MTC_decile,MTC_RR,SD_decile,SD_RR,SC_decile,SC_RR
66008,51,Percent of population within 1/2 mile of majo...,2012.0,9.0,Total,RE,14.0,Southern California,NaN,NaN,Southern California,14.0,6350660.0,18051534.0,0.351807,0.351587,0.352028,0.000112,0.031948,NaN,NaN,##############################################...,NaN,NaN,NaN,NaN,NaN,1.000021


**data_mod2:** removing columns that repeat the same value

In [5]:
data_mod2 = data_mod1.copy()

# Each value is the same for 'ind_id' and 'ind_definition' columns
print(data_mod2['ind_id'].unique())
print(data_mod2['ind_definition'].unique())

['51' 'END OF TABLE']
['Percent of population residing within ½ mile of a major transit stop'
 nan ' Percent of population within 1/2 mile of major transit stop']


In [6]:
# Dropping the columns
data_mod2 = data_mod2.drop(columns = {'ind_id', 'ind_definition'})
data_mod2.head(1)

,reportyear,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,region_name,region_code,pop_trans_acc,pop2010,p_trans_acc,LL_95CI,UL_95CI,se,rse,SAC_decile,SAC_RR,version,MTC_decile,MTC_RR,SD_decile,SD_RR,SC_decile,SC_RR
0,2008.0,3.0,AfricanAm,CO,6061.0,Placer,Placer,6061.0,Sacramento Area,8.0,55.0,4427.0,0.012424,0.009161,0.015687,0.001665,13.399974,NaN,0.07046,26Nov2013,NaN,NaN,NaN,NaN,NaN,NaN


**data_mod3:** changing 'reportyear', 'race_eth_code', 'geotypevalue', 'county_fips', 'region_code', 'pop_trans_acc', & 'pop2010' columns from type **float to int**

In [7]:
data_mod3 = data_mod2.copy()

# Missing values for each column (used below)
missing_values = data_mod3.isna().sum()

# Changing to type 'Int'
data_mod3['reportyear'] = pd.array(data_mod3['reportyear'], dtype=pd.Int64Dtype())
data_mod3['race_eth_code'] = pd.array(data_mod3['race_eth_code'], dtype=pd.Int64Dtype())
data_mod3['geotypevalue'] = pd.array(data_mod3['geotypevalue'], dtype=pd.Int64Dtype())
data_mod3['county_fips'] = pd.array(data_mod3['county_fips'], dtype=pd.Int64Dtype())
data_mod3['region_code'] = pd.array(data_mod3['region_code'], dtype=pd.Int64Dtype())
data_mod3['pop_trans_acc'] = pd.array(data_mod3['pop_trans_acc'], dtype=pd.Int64Dtype())
data_mod3['pop2010'] = pd.array(data_mod3['pop2010'], dtype=pd.Int64Dtype())

data_mod3.head(1)

,reportyear,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,region_name,region_code,pop_trans_acc,pop2010,p_trans_acc,LL_95CI,UL_95CI,se,rse,SAC_decile,SAC_RR,version,MTC_decile,MTC_RR,SD_decile,SD_RR,SC_decile,SC_RR
0,2008,3,AfricanAm,CO,6061,Placer,Placer,6061,Sacramento Area,8,55,4427,0.012424,0.009161,0.015687,0.001665,13.399974,NaN,0.07046,26Nov2013,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Checking that it retained the missing values
missing_values_mod1 = data_mod3.isna().sum()
missing_values_mod1 == missing_values

reportyear       True
race_eth_code    True
race_eth_name    True
geotype          True
geotypevalue     True
geoname          True
county_name      True
county_fips      True
region_name      True
region_code      True
pop_trans_acc    True
pop2010          True
p_trans_acc      True
LL_95CI          True
UL_95CI          True
se               True
rse              True
SAC_decile       True
SAC_RR           True
version          True
MTC_decile       True
MTC_RR           True
SD_decile        True
SD_RR            True
SC_decile        True
SC_RR            True
dtype: bool

**data_mod4**: examining the 'version' column

In [9]:
data_mod4 = data_mod3.copy()
data_mod4['version']

0                                                26Nov2013
1                                                26Nov2013
2                                                26Nov2013
3                                                26Nov2013
4                                                26Nov2013
                               ...                        
66004    ##############################################...
66005    ##############################################...
66006    ##############################################...
66007    ##############################################...
66008    ##############################################...
Name: version, Length: 66009, dtype: object

_The 'version ' column contains a mix of 'DD/M/YYYY' & '######...' values. Remove the version column, as we already have Year values._

In [10]:
data_mod4 = data_mod4.drop(columns = {'version'})

**data_mod5**: renaming 'reportyear' column to 'year'

In [11]:
data_mod5 = data_mod4.copy()

data_mod5 = data_mod5.rename(
    columns = {'reportyear': 'year',}
)

data_mod5.head(1)

,year,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,region_name,region_code,pop_trans_acc,pop2010,p_trans_acc,LL_95CI,UL_95CI,se,rse,SAC_decile,SAC_RR,MTC_decile,MTC_RR,SD_decile,SD_RR,SC_decile,SC_RR
0,2008,3,AfricanAm,CO,6061,Placer,Placer,6061,Sacramento Area,8,55,4427,0.012424,0.009161,0.015687,0.001665,13.399974,NaN,0.07046,NaN,NaN,NaN,NaN,NaN,NaN


**data_mod6**: removing the rows with Year absent

In [12]:
# Looking at how missing values for 'reportyear'/'Year' correlates with other variables
missing_values

reportyear           3
race_eth_code        3
race_eth_name        3
geotype              3
geotypevalue         3
geoname             57
county_name         66
county_fips         66
region_name          3
region_code          3
pop_trans_acc     1185
pop2010              3
p_trans_acc       1560
LL_95CI           1185
UL_95CI           1560
se                1185
rse              14703
SAC_decile       65496
SAC_RR           61434
version              3
MTC_decile       64208
MTC_RR           49926
SD_decile        65329
SD_RR            59942
SC_decile        61716
SC_RR            28285
dtype: int64

There are **3 rows** where a missing Year value corresponds to a missing value for other variables. 

Specifically, 'race_eth_code', 'race_eth_name', 'geotype', 'geotypevalue', 'region_name', 'region_code', 'pop2010', 'version'. Since these are important characteristics and the data set is very large, we can remove the 3 rows (<0.01% of the overall data).

In [13]:
data_mod6 = data_mod5.copy()
data_mod6 = data_mod6.dropna(subset=['year'])

# there should be 3 less rows
len(data_mod5) - len(data_mod6)

3

### Exporting to 'tidy-data'

**Final dataset**

In [14]:
data_mod6.to_csv('tidy-data')